<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="../figs/uoc_masterbrand_3linies_positiu.png", align="left">
    </div>
    <div style="float: right; width: 50%;">
        <p style="margin: 0; padding-top: 22px; text-align:right;">M2.855 · Models avançats de mineria de dades</p>
        <p style="margin: 0; text-align:right;">Màster universitari en Ciència de dades (<i>Data science</i>)</p>
        <p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
    </div>
</div>
<div style="width:100%;">&nbsp;</div>

# Xarxa neuronal per a classificació de FASHION-MNIST

En aquest exemple veurem la creació d'una xarxa neuronal mitjançant l'ús de Python i la llibreria `keras`, per fer una classificació d'objectes a partir del conjunt de dades [Fashion-MNITS](http://yann.lecun.com/exdb/mnist/). 

## 1. Càrrega de llibreries i dades

In [ ]:
import numpy as np
import keras
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization
import matplotlib.pyplot as plt

Descarrega el conjunt de dades MNIST i fes la partició entrenament / test.

In [ ]:
(x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = fashion_mnist.load_data()

Visualització d'un dels objectes per verificar de manera visual la mida, el canal de color, etc.

In [ ]:
first_image = x_train_orig[0]
first_image = np.array(first_image, dtype='float')
pixels = first_image.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

Realitzem una conversió de format perquè les imatges estiguin correctament emmagatzemades a l'estructura de 4 dimensions:
- número d'imatge 
- alçada 
- amplada 
- canal de color (en aquest cas concret, un sol canal que correspon a l'escala de gris).

A continuació, realitzem la conversió a variable categòrica de les etiquetes de les imatges mitjançant la funció `to_categoricaly`.

In [ ]:
x_train = x_train_orig.reshape(60000,28,28,1)
x_test = x_test_orig.reshape(10000,28,28,1)

y_train = to_categorical(y_train_orig, 10)
y_test = to_categorical(y_test_orig, 10)

## 2. Creació del model

A continuació construïm la xarxa neuronal, que en aquest cas estarà formada per les capes següents:
- Capa convolucional de 32 *kernels* de mida *3x3* i activació `RELU`
- Capa convolucional de 64 *kernels* de mida *3x3* i activació `RELU`
- Capa *Max pooling*
- Capa *Dropout* amb probabilitat igual al 25 %
- Capa densament connectada amb 128 neurones i activació `RELU`
- Capa *Dropout*
- Capa de sortida, formada per 10 neurones (tenim 10 classes al conjunt de dades) i *softmax*

Finalment, veiem els detalls del model construït (per a cada capa, veiem les dimensions de sortida, el nombre de paràmetres entrenables, etc.).

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))

Resum de la informació del model.

In [ ]:
model.summary()

Realitzem l'entrenament sobre el conjunt de *train*.

In [ ]:
batch_size = 128
n_epochs = 12

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

mfit = model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          epochs=n_epochs)

## 3. Avaluació del model

Revisem la informació disponible de l'entrenament per veure'n l'evolució.

In [ ]:
print(mfit.history.keys())

Creem dos gràfics:
1. Al primer presentem l'evolució de l'*accuracy* del model durant les 12 èpoques de l'entrenament, tenint en compte les dades d'entrenament i test.
2. Al segon presentem l'evolució de la funció de *loss* durant l'entrenament.

In [ ]:
# Accuracy
plt.plot(mfit.history['accuracy'])
plt.plot(mfit.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Loss
plt.plot(mfit.history['loss'])
plt.plot(mfit.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Realitzem l'avaluació del model sobre el conjunt de dades de *test*.

In [ ]:
# Avaluació del model sobre el conjunt de test
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss    :', score[0])
print('Test accuracy:', score[1])

En primer lloc, mostrem el vector de prediccions per a cadascun dels 4 primers registres de test. 

Cada vector està format per 10 números, que indiquen la probabilitat d'aquest registre de pertànyer a cadascuna de les 10 categories que hi ha en aquest conjunt de dades.

En segon lloc, mostrem els valors correctes (etiquetats al conjunt original) per a aquests 4 primers registres corresponents a les quatre primeres imatges del conjunt de test.

In [ ]:
# Predicció de les quatre primeres imatges del conjunt de test
print(model.predict(x_test[:4]))

In [ ]:
y_test[:4]

En segon lloc, mostrem els valors correctes (etiquetats al conjunt original) per a aquests 4 primers registres.